In [11]:
import os
import numpy as np
import cv2

In [12]:
# img_dir = '../images/'
img_dir = '../4_images/'

In [13]:
all_image_names = ['image_' + str(i) for i in range(118)]
ext_dict = {
    'og': 'original/',
    'all': 'all_edges/',
    '0': '0/',
    '30': '30/',
    '60': '60/',
    '90': '90/',
    '120': '120/',
    '150': '150/'
}
orientation_angle_dict = {
    '0': 0,  # 0 degrees
    '30': np.pi / 6,  # 30 degrees
    '60': np.pi / 3,  # 60 degrees
    '90': np.pi / 2,  # 90 degrees
    '120': 2 * np.pi / 3,  # 120 degrees
    '150': 5 * np.pi / 6  # 150 degrees
}
orientations = [0, 30, 60, 90, 120, 150]

In [14]:
def sum_intensity(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.flatten()
    img = np.array(img)
    img = img.astype('float32')
    img /= 255
    return img.mean()

In [15]:
# Dict that maps each image to each orientation to each score
edge_scores_dict = {}
# For each image...
for img in all_image_names:
    if '57' in img or '61' in img or '81' in img or '44' in img:
        # Initialize the edge score dict
        edge_scores_dict[img] = {}
        # Get path to output with all edges
        all_edges_path = img_dir + ext_dict['all'] + str(img) + '.png'
        # Populate edge score for all edges as baseline
        all_edges_score = sum_intensity(all_edges_path)
        edge_scores_dict[img]['all'] = all_edges_score
        # For each orientation...
        for o in orientations:
            orientation = str(o)
            # Get path to output with edges in this orientation
            curr_orientation_path = img_dir + ext_dict[orientation] + str(img) + '.png'
            # Compute score for this orientation
            curr_orientation_score = sum_intensity(curr_orientation_path)
            # Store in dict
            edge_scores_dict[img][orientation] = curr_orientation_score

In [16]:
edge_scores_dict

{'image_44': {'all': 0.009430916,
  '0': 0.00051311456,
  '30': 0.0003999139,
  '60': 0.00055394106,
  '90': 0.0008016835,
  '120': 0.0013417065,
  '150': 0.00095292705},
 'image_57': {'all': 0.0072986605,
  '0': 0.0018872967,
  '30': 0.0003191888,
  '60': 0.00033217907,
  '90': 0.0003813564,
  '120': 0.0008220968,
  '150': 0.00082302466},
 'image_61': {'all': 0.018846987,
  '0': 0.000398986,
  '30': 0.00067178113,
  '60': 0.0011264395,
  '90': 0.0016581117,
  '120': 0.0035982972,
  '150': 0.0020867896},
 'image_81': {'all': 0.024785383,
  '0': 0.0008749856,
  '30': 0.0009102448,
  '60': 0.0018817295,
  '90': 0.007973225,
  '120': 0.0014910942,
  '150': 0.00055115746}}

In [17]:
# Normalize edge_scroes_dict 
normalized_edge_scores_dict = {}
# For each image...
for img in all_image_names:
    if '57' in img or '61' in img or '81' in img or '44' in img:
        # Initialize the edge score dict
        normalized_edge_scores_dict[img] = {}
        # Get score for all edges
        all_edges_score = edge_scores_dict[img]['all']
        # For each orientation...
        for o in orientations:
            orientation = str(o)
            # Get score for this orientation
            curr_orientation_score = edge_scores_dict[img][orientation]
            # Normalize by all edges score
            normalized_edge_scores_dict[img][orientation] = curr_orientation_score / all_edges_score

In [18]:
# normalized_edge_scores_dict

In [19]:
# Construct a dataframe from normalized_edge_scores_dict, where each row is an image and each column is an orientation
import pandas as pd
df = pd.DataFrame(normalized_edge_scores_dict)
df = df.transpose()
df.columns = orientations
# add a row for the average of each column
df.loc['mean'] = df.mean()
df.loc['variance'] = df.var()
df

,0,30,60,90,120,150
image_44,0.054408,0.042405,0.058737,0.085006,0.142267,0.101043
image_57,0.258581,0.043733,0.045512,0.052250,0.112637,0.112764
image_61,0.021170,0.035644,0.059768,0.087978,0.190922,0.110723
image_81,0.035302,0.036725,0.075921,0.321691,0.060160,0.022237
mean,0.092365,0.039627,0.059984,0.136731,0.126496,0.086692
variance,0.009348,0.000012,0.000116,0.011600,0.002248,0.001404


In [20]:
# save df to csv
# df.to_csv('../data/normalized_edge_scores.csv')
df.to_csv('../data/4_images_normalized_edge_scores.csv')